In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Extract CSVs into DataFrames

In [2]:
pizza_file = "Resources/Pizza_data.csv"
pizza_one_df = pd.read_csv(pizza_file)
pizza_one_df.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVz3Y-7h3D1zeR_xDAqm,2017-06-30T05:05:40Z,2019-05-01T15:43:09Z,4203 E Kiehl Ave,"Pizza,Restaurant,American restaurants,Pizza Pl...",Accommodation & Food Services,Sherwood,US,us/ar/sherwood/4203ekiehlave/-1051391616,34.832300,...,USD,"2018-05-01T04:25:37.197Z,2018-04-16T04:36:02.3...",NaN,Cheese Pizza,Shotgun Dans Pizza,72120,USD,0,25,AR
1,AVweGPFF_7pvs4fzAAzQ,2016-04-02T04:02:49Z,2019-05-01T15:27:50Z,25 E Camelback Rd,"Pizza,Pizza Place,Restaurants",Accommodation & Food Services,Phoenix,US,us/az/phoenix/25ecamelbackrd/-727422936,33.509266,...,USD,"2018-03-03T02:38:06.381Z,2018-01-18T20:18:10.0...",NaN,Pizza Cookie,Sauce Pizza Wine,85012,USD,0,25,AZ
2,AVwdRGa9_7pvs4fz4E3K,2016-03-03T18:39:49Z,2019-05-01T12:52:25Z,3703 Paxton Ave,"Restaurant,Pizza Place,Restaurants",Accommodation & Food Services,Cincinnati,US,us/oh/cincinnati/3703paxtonave/-619797122,39.144883,...,USD,"2018-04-10T07:58:34.585Z,2018-04-21T05:43:21.4...",a saucelessampcomma double cheese pizza with a...,Pizza Blanca,Mios Pizzeria,45209,USD,0,25,OH
3,AVwdX4psIN2L1WUfvJB1,2016-03-29T05:08:59Z,2019-05-01T12:52:20Z,30495 John R Rd,"Pizza,Carry-out food,Pizza Place,Restaurants",Accommodation & Food Services,Madison Heights,US,us/mi/madisonheights/30495johnrrd/-874863116,42.516669,...,USD,"2016-10-20T21:50:02Z,2016-03-29T05:08:59Z",NaN,Small Pizza,Hungry Howies Pizza,48071,USD,25,40,MI
4,AVwdaeTtkufWRAb55pSH,2016-03-31T02:34:04Z,2019-05-01T12:50:45Z,3600 Eastern Ave,"Pizza,American restaurants,Pizza Place,Pizza e...",Accommodation & Food Services,Baltimore,US,us/md/baltimore/3600easternave/-1270965359,39.286630,...,USD,2016-03-31T02:34:04Z,NaN,Pizza Sub,Spartan Pizzeria,21224,USD,0,25,MD


In [3]:
pizza_two_file = "Resources/pizza_two.csv"
pizza_two_df = pd.read_csv(pizza_two_file)
pizza_two_df.head()

,id,address,categories,city,country,keys,latitude,longitude,menuPageURL,menus.amountMax,...,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province
0,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Pizza Place,Bend,US,us/or/bend/cascadevillagemallacrossfromtarget/...,44.102665,-121.300797,NaN,22.50,...,USD,2016-03-31T02:19:42Z,NaN,Bianca Pizza,Little Pizza Paradise,97701,NaN,NaN,NaN,OR
1,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Pizza Place,Bend,US,us/or/bend/cascadevillagemallacrossfromtarget/...,44.102665,-121.300797,NaN,18.95,...,USD,2016-03-31T02:19:42Z,NaN,Cheese Pizza,Little Pizza Paradise,97701,NaN,NaN,NaN,OR
2,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,12.00,...,USD,2015-10-23T03:57:26Z,NaN,"Pizza, Margherita",The Brentwood,90049,USD,50.0,55.0,Brentwood
3,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,13.00,...,USD,2015-10-23T03:57:26Z,NaN,"Pizza, Mushroom",The Brentwood,90049,USD,50.0,55.0,Brentwood
4,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,"American Restaurant,Bar,Bakery",Los Angeles,US,us/brentwood/losangeles/148sbarringtonave/-151...,34.064563,-118.469017,NaN,13.00,...,USD,2015-10-23T03:57:26Z,"Olives, onions, capers, tomatoes","Pizza, Puttenesca",The Brentwood,90049,USD,50.0,55.0,Brentwood


### Transform pizza two DataFrame

In [4]:
# Create a filtered dataframe from specific columns
pizza_two_cols = ["id", "address", "city", "latitude", "longitude", "menus.amountMax", "name", "menus.name"]
pizza_two_transformed= pizza_two_df[pizza_two_cols].copy()

# Rename the column headers
pizza_two_transformed = pizza_two_transformed.rename(columns={"id": "id",
                                                          "address": "address",
                                                          "city": "city",
                                                           "latitude": "latitude",
                                                             "longitude": "longitude",
                                                             "menus.amountMax": "price",
                                                             "name": "restaurant",
                                                             "menus.name": "pizza_type"})

# Clean the data by dropping duplicates and setting the index
pizza_two_transformed.drop_duplicates("id", inplace=True)
#pizza_two_transformed.set_index("key", inplace=True)
pizza_two_transformed = pizza_two_transformed.dropna()

pizza_two_transformed.head()

,id,address,city,latitude,longitude,price,restaurant,pizza_type
0,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Bend,44.102665,-121.300797,22.50,Little Pizza Paradise,Bianca Pizza
2,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,Los Angeles,34.064563,-118.469017,12.00,The Brentwood,"Pizza, Margherita"
7,AVwc_7d_IN2L1WUfrKNN,5142 Hollywood Blvd,Los Angeles,34.101742,-118.301973,10.99,Bravo Pizza Hollywood,Cheese Pizza
13,AVwc_8WAByjofQCxkdDA,801 Saint Emanuel St,Houston,29.752479,-95.354164,9.95,Lucky's Pub,Create Your Own Pizza
14,AVwc_abAIN2L1WUfrEps,478 South St,Hyannis,41.648278,-70.291345,8.95,Roadhouse Cafe,Cheese Pizza


In [5]:
#pizza_two_transformed.reset_index()

### Transform Pizza One DataFrame

In [6]:
pizza_one_cols = ["id", "address", "city", "latitude", "longitude", "priceRangeMax", "name", "menus.name" ]
pizza_one_transformed = pizza_one_df[pizza_one_cols].copy()

# Rename the column headers
pizza_one_transformed = pizza_one_transformed.rename(columns={"id": "id",
                                                         "address": "address",
                                                         "city": "city",
                                                         "latitude": "latitude",
                                                        "longitude": "longitude",
                                                             "menus.name": "pizza_type",
                                                             "priceRangeMax": "price",
                                                             "name": "restaurant"})

# Set index
pizza_one_transformed.drop_duplicates("id", inplace=True)
#pizza_one_transformed.set_index("key", inplace=True)
pizza_one_transformed = pizza_one_transformed.dropna()

#pizza_one_transformed.reset_index()

In [7]:
#Join tables on ID
df = pizza_one_transformed.append([pizza_two_transformed])

# dropping ALL duplicte values 
df.drop_duplicates("id", inplace = True)      
df.count()

id            3115
address       3115
city          3115
latitude      3115
longitude     3115
price         3115
restaurant    3115
pizza_type    3115
dtype: int64

### Create database connection

In [8]:
connection_string = "postgres:waterbottle@localhost:5432/new_pizza_db"
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
# Confirm tables
engine.table_names()

['pizza', 'pizza_one', 'pizza_two']

### Load DataFrames into database

In [10]:
pizza_one_transformed.to_sql(name='pizza_one', con=engine, if_exists='append', index=False)

In [11]:
pizza_two_transformed.to_sql(name='pizza_two', con=engine, if_exists='append', index=False)

In [12]:
df.to_sql(name='pizza', con=engine, if_exists='append', index=False)

In [13]:
pd.read_sql_query('select * from pizza_one', con=engine).head()

,id,address,city,latitude,longitude,price,restaurant,pizza_type
0,AVz3Y-7h3D1zeR_xDAqm,4203 E Kiehl Ave,Sherwood,34.832300,-92.183800,25.0,Shotgun Dans Pizza,Cheese Pizza
1,AVweGPFF_7pvs4fzAAzQ,25 E Camelback Rd,Phoenix,33.509266,-112.073044,25.0,Sauce Pizza Wine,Pizza Cookie
2,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,Cincinnati,39.144883,-84.432685,25.0,Mios Pizzeria,Pizza Blanca
3,AVwdX4psIN2L1WUfvJB1,30495 John R Rd,Madison Heights,42.516669,-83.106630,40.0,Hungry Howies Pizza,Small Pizza
4,AVwdaeTtkufWRAb55pSH,3600 Eastern Ave,Baltimore,39.286630,-76.566984,25.0,Spartan Pizzeria,Pizza Sub


In [14]:
pd.read_sql_query('select * from pizza_two', con=engine).head()

,id,address,city,latitude,longitude,price,restaurant,pizza_type
0,AVwc_6KEIN2L1WUfrKAH,Cascade Village Mall Across From Target,Bend,44.102665,-121.300797,22.50,Little Pizza Paradise,Bianca Pizza
1,AVwc_6qRByjofQCxkcxw,148 S Barrington Ave,Los Angeles,34.064563,-118.469017,12.00,The Brentwood,"Pizza, Margherita"
2,AVwc_7d_IN2L1WUfrKNN,5142 Hollywood Blvd,Los Angeles,34.101742,-118.301973,10.99,Bravo Pizza Hollywood,Cheese Pizza
3,AVwc_8WAByjofQCxkdDA,801 Saint Emanuel St,Houston,29.752479,-95.354164,9.95,Lucky's Pub,Create Your Own Pizza
4,AVwc_abAIN2L1WUfrEps,478 South St,Hyannis,41.648278,-70.291345,8.95,Roadhouse Cafe,Cheese Pizza


In [15]:
pd.read_sql_query('select * from pizza', con=engine).head()

,id,address,city,latitude,longitude,price,restaurant,pizza_type
0,AVz3Y-7h3D1zeR_xDAqm,4203 E Kiehl Ave,Sherwood,34.832300,-92.183800,25.0,Shotgun Dans Pizza,Cheese Pizza
1,AVweGPFF_7pvs4fzAAzQ,25 E Camelback Rd,Phoenix,33.509266,-112.073044,25.0,Sauce Pizza Wine,Pizza Cookie
2,AVwdRGa9_7pvs4fz4E3K,3703 Paxton Ave,Cincinnati,39.144883,-84.432685,25.0,Mios Pizzeria,Pizza Blanca
3,AVwdX4psIN2L1WUfvJB1,30495 John R Rd,Madison Heights,42.516669,-83.106630,40.0,Hungry Howies Pizza,Small Pizza
4,AVwdaeTtkufWRAb55pSH,3600 Eastern Ave,Baltimore,39.286630,-76.566984,25.0,Spartan Pizzeria,Pizza Sub
